In [ ]:
from faker import Faker
import numpy as np

def gerar_base_historica_ficticia():

    # Inicializa o gerador de dados falsos
    faker = Faker('pt_BR')
    
    # Define o número de linhas que você quer gerar
    n_pessoas = 1000
    
    # Gera os dados de pessoas com municípios
    dados_pessoas = {
        'uf': np.random.choice(['SP', 'RJ', 'MG', 'RS', 'BA'], size=n_pessoas),
        'municipio': [faker.city() for _ in range(n_pessoas)],
        'nome': [faker.name() for _ in range(n_pessoas)],
        'data_inicio': [faker.date_between(start_date='-5y', end_date='today') for _ in range(n_pessoas)],
        'data_fim': [faker.date_between(start_date='today', end_date='+5y') for _ in range(n_pessoas)],
    }
    
    # Cria o DataFrame de pessoas
    df_pessoas = pd.DataFrame(dados_pessoas)
    
    # Converte para datetime
    df_pessoas['data_inicio'] = pd.to_datetime(df_pessoas['data_inicio'])
    df_pessoas['data_fim'] = pd.to_datetime(df_pessoas['data_fim'])
    
    # Garante que data_inicio <= data_fim
    df_pessoas.loc[df_pessoas['data_inicio'] > df_pessoas['data_fim'], ['data_inicio', 'data_fim']] = \
        df_pessoas.loc[df_pessoas['data_inicio'] > df_pessoas['data_fim'], ['data_fim', 'data_inicio']].values

    return df_pessoas

    

In [ ]:
import os
from typing import List
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds

def criar_diretorio(caminho: str) -> None:
    """Garante que o diretório existe."""
    os.makedirs(caminho, exist_ok=True)

def converter_para_tabela(df: pd.DataFrame) -> pa.Table:
    """Converte um DataFrame Pandas para uma Tabela PyArrow."""
    return pa.Table.from_pandas(df, preserve_index=False)

def escrever_dataset_particionado(
    tabela: pa.Table,
    caminho_base: str,
    particoes: List[str]
) -> None:
    """Escreve a tabela particionada em Parquet."""
    ds.write_dataset(
        data=tabela,
        base_dir=caminho_base,
        format="parquet",
        partitioning=particoes,
        existing_data_behavior="overwrite_or_ignore"
    )

def gerar_arvore_bruta_particionada(
    df: pd.DataFrame,
    colunas_particao: List[str],
    coluna_data_inicio: str,
    coluna_data_fim: str,    
    caminho_destino: str
) -> None:
    """
    Gera uma árvore de arquivos Parquet particionados pelas colunas fornecidas.
    """
    if not colunas_particao:
        raise ValueError("É necessário informar ao menos uma coluna de partição.")

    # Cria a pasta raiz
    criar_diretorio(caminho_destino)

    # Seleciona as colunas essenciais
    colunas_necessarias = set(colunas_particao + [coluna_data_inicio, coluna_data_fim])
    colunas_disponiveis = set(df.columns)

    if not colunas_necessarias.issubset(colunas_disponiveis):
        faltando = colunas_necessarias - colunas_disponiveis
        raise ValueError(f"Colunas faltando no DataFrame: {faltando}")

    df = df.copy()
    df[coluna_data_inicio] = pd.to_datetime(df[coluna_data_inicio])
    df[coluna_data_fim] = pd.to_datetime(df[coluna_data_fim])

    tabela = converter_para_tabela(df)

    escrever_dataset_particionado(
        tabela=tabela,
        caminho_base=caminho_destino,
        particoes=colunas_particao
    )

df_pessoas = gerar_base_historica_ficticia()

gerar_arvore_bruta_particionada(
    df=df_pessoas,
    colunas_particao=['uf', 'municipio'],
    coluna_data_inicio='data_inicio',
    coluna_data_fim='data_fim',    
    caminho_destino="../../dados/dados_brutos_iceberg"
)